# Classification with multiple Approaches
* Mohammad Hassan Heydari

## 1. Classification with Neural Networks

**Importing necessary libraries and checking platform**

In [14]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
import jax
import pandas as pd

In [10]:
device = jax.devices()[0]
print(f'Training platform : {device.platform}')
print(f'Device Name : {device.device_kind}')


Training platform : gpu
Device Name : NVIDIA GeForce MX450


**Loading data**

In [19]:
leaves_csv = pd.read_csv('Data/leaves.csv')

leaves_csv = leaves_csv.sample(frac=1).reset_index(drop=True)

leaves_np = jnp.asarray(leaves_csv)
print(f'shape of leaves: {leaves_np.shape}')

shape of leaves: (339, 16)


In [18]:
# first column is targets, other columns are features, so :
targets = leaves_np[:, 0]
features = leaves_np[:, 1:]
print(f'shape of features: {features.shape}')
print(f'shape of targets: {targets.shape}')
print(f'first 10 elements of targets array: {targets[0:10]}')

shape of features: (339, 15)
shape of targets: (339,)
first 10 elements of targets array: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [11]:
def random_layer_params(m, n, key, scale=1e-2):
  w_key, b_key = random.split(key)
  return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

# Initialize all layers for a fully-connected neural network with sizes "sizes"
def init_network_params(sizes, key):
  keys = random.split(key, len(sizes))
  return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]